In [2]:
# 이름이 약간 다르더라도(대소문자, 스페이스, 오타 등) 가장 유사한 향수 이름을 매칭하기 위한 라이브러리
# !pip install thefuzz[speedup] pandas
!pip install "thefuzz[speedup]" # mac zsh version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [thefuzz]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
# csv 파일 인코딩 정보 확인 위함
!pip install chardet


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import chardet

with open("./dataset/final_perfume_data.csv", "rb") as f:
    rawdata = f.read(10000)  # 파일 앞부분 10KB만 검사
    result = chardet.detect(rawdata)
    print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [1]:
import pandas as pd

for enc in ['utf-8-sig', 'utf-8', 'cp949', 'euc-kr', 'latin1']:
    try:
        # df1 = pd.read_csv("./dataset/final_perfume_data.csv", encoding=enc)
        df2 = pd.read_csv("./dataset/perfume.csv", encoding=enc)
        print(f"Success with encoding: {enc}")
        break
    except Exception as e:
        print(f"Failed with encoding: {enc}, error: {e}")
        print("\n")
        
        
# df1 : Success with encoding: latin1
# df2 : Success with encoding: utf-8-sig
        

Success with encoding: utf-8-sig


In [13]:
import pandas as pd
import re
from thefuzz import process

# 1. CSV 불러오기 (+ 인코딩)
# description , 이미지 있는 데이터
# Perfume Recommendation Dataset => https://www.kaggle.com/datasets/nandini1999/perfume-recommendation-dataset?utm_source=chatgpt.com
df1 = pd.read_csv("./dataset/final_perfume_data.csv", encoding="latin1")   # Perfume Recommendation Dataset
# 농도 메인어코드 탑/미들/베이스노트 
# Parfumo Fragrance Dataset => https://www.kaggle.com/datasets/olgagmiufana1/parfumo-fragrance-dataset
df2 = pd.read_csv("./dataset/parfumo_datos.csv", encoding="utf-8-sig")   # Parfumo Fragrance Dataset
# 더 필요한 칼럼 : 가격 용량 부향률 성별

# 2. 번호/특수문자 제거 함수
def clean_name(name):
    if pd.isna(name):
        return ""
    # "#숫자 " 패턴 제거 + 소문자 변환 + 앞뒤 공백 제거
    name = re.sub(r"^#\d+\s*", "", str(name))
    name = name.lower().strip()
    return name

# 3. 전처리 컬럼 생성
df1["Name_clean"] = df1["Name"].apply(clean_name)
df2["Name_clean"] = df2["Name"].apply(clean_name)

# 4. fuzzy matching 함수 수정
def match_name(name, choices, threshold=85):
    result = process.extractOne(name, choices, score_cutoff=threshold)
    if result is None:
        return None
    match_name, score = result[0], result[1]
    return match_name

# 5. 매칭 수행
matches = []
for name in df1["Name_clean"]:
    matched_name = match_name(name, df2["Name_clean"], threshold=85)
    matches.append(matched_name)

df1["Matched_Name_clean"] = matches

# 6. 병합
df_merged = pd.merge(df1, df2, left_on="Matched_Name_clean", right_on="Name_clean", how="left", suffixes=("_kaggle", "_parfumo"))

# 7. 결과 저장
df_merged.to_csv("dataset/perfume_merged_fuzzy.csv", index=False)

print(f"병합된 데이터셋 크기: {df_merged.shape}") # 3.3MB
print(df_merged[["Name_kaggle", "Matched_Name_clean", "Name_parfumo"]].head(10))


병합된 데이터셋 크기: (2218, 20)
                            Name_kaggle  \
0                  Tihota Eau de Parfum   
1                           Sola Parfum   
2                        Kagiroi Parfum   
3          Velvet Fantasy Eau de Parfum   
4   A Blvd. Called Sunset Eau de Parfum   
5  Freckled and Beautiful Eau de Parfum   
6           Exit the King Eau de Parfum   
7                          Eshu Extrait   
8                    Saringkarn Extrait   
9                        Arsalan Parfum   

                                  Matched_Name_clean  \
0                   méditation de la lune le ré noir   
1                         parfum exaltant le ré noir   
2                         parfum exaltant le ré noir   
3   #flower power ramón monegal 2010 eau de toilette   
4  on a clear day you can see forever cb i hate p...   
5  *cough cough* i'm sick. sixteen92 2024 extrait...   
6  - havana, glass of vanilla cocktail on the bea...   
7  *cough cough* i'm sick. sixteen92 2024 extrait...  

In [7]:
import pandas as pd
import re

# 0) CSV 읽기 (인코딩은 환경에 맞게 조정: 보통 'utf-8-sig' 또는 'cp949')
df = pd.read_csv("./dataset/perfume.csv", encoding="utf-8-sig")  # 필요시 cp949로 변경

# ---------------------------------------------------------------------
# 유틸
# ---------------------------------------------------------------------
def _normalize_newlines(s: str) -> str:
    # \r\n, \r -> \n 통일 + 리터럴 "\n" 도 실제 개행으로 변환
    return s.replace("\r\n", "\n").replace("\r", "\n").replace("\\n", "\n")

def _cleanup_bullets_and_whitespace(s: str) -> str:
    """
    각 줄 맨앞의 '-', '*' 같은 불릿 제거 후 줄을 공백으로 합쳐 한 줄로 만듭니다.
    """
    lines = []
    for line in s.split("\n"):
        line = re.sub(r"^\s*[-*]\s*", "", line)  # 앞의 불릿 제거
        if line.strip():
            lines.append(line.strip())
    one_line = " ".join(lines)
    one_line = re.sub(r"\s+", " ", one_line).strip()
    return one_line

def _normalize_list_string(s: str) -> str:
    """
    '프루티 / 스위트 / 레더' 같은 문자열을
    '프루티, 스위트, 레더' 형태로 통일.
    """
    # 슬래시, 쉼표(전각 포함) 등으로 분리
    parts = [p.strip() for p in re.split(r"[\/,、\u3001]+", s) if p.strip()]
    return ", ".join(parts)

def _extract_section(text: str, pattern: str, take_to_end=False) -> str:
    """
    [헤더]로 시작하는 섹션을 캡처합니다.
    - take_to_end=True 이면 문서 끝까지, False면 다음 [헤더] 직전까지.
    pattern: 예) r"\[부향률\]"
    """
    if take_to_end:
        m = re.search(pattern + r"\s*(.+)\Z", text, flags=re.S)
    else:
        m = re.search(pattern + r"\s*(.+?)(?=\n\[[^\]]+\]|\Z)", text, flags=re.S)
    return m.group(1).strip() if m else ""

def _extract_note_line(block: str, label_kr: str) -> str:
    """
    메인 노트 블록에서 '- 탑 노트: ...' 처럼 특정 라벨 줄만 뽑아 정리.
    반환은 'A, B, C' 형태 문자열.
    """
    # 줄 단위 탐색 (불릿/공백 허용, 콜론 형태 허용)
    mm = re.search(
        rf"(?m)^\s*-\s*{label_kr}\s*노트\s*[:：]\s*(.+?)\s*$",
        block
    )
    if not mm:
        return ""
    raw = mm.group(1).strip()
    return _normalize_list_string(raw)

# ---------------------------------------------------------------------
# 메인 파서
# ---------------------------------------------------------------------
def parse_description(text):
    if not isinstance(text, str):
        text = ""
    text = _normalize_newlines(text)

    out = {
        "부향률": "",
        "메인 어코드": "",
        "탑 노트": "",
        "미들 노트": "",
        "베이스 노트": "",
        "향 설명": "",
    }

    # [부향률] : 다음 [헤더] 전까지
    sec = _extract_section(text, r"\[부향률\]")
    if sec:
        out["부향률"] = _cleanup_bullets_and_whitespace(sec)

    # [메인 어코드] : 다음 [헤더] 전까지 -> 리스트 정규화 후 "A, B, C"
    sec = _extract_section(text, r"\[메인\s*어코드\]")
    if sec:
        sec = _cleanup_bullets_and_whitespace(sec)
        out["메인 어코드"] = _normalize_list_string(sec)

    # [메인 노트] 블록
    notes_block = _extract_section(text, r"\[메인\s*노트\]")
    if notes_block:
        out["탑 노트"]    = _extract_note_line(notes_block, "탑")
        out["미들 노트"]  = _extract_note_line(notes_block, "미들")
        out["베이스 노트"] = _extract_note_line(notes_block, "베이스")

    # [향 설명] : 문서 끝까지 포함 (하단 * 안내문까지 포함)
    sec = _extract_section(text, r"\[향\s*설명\]", take_to_end=True)
    if sec:
        out["향 설명"] = _cleanup_bullets_and_whitespace(sec)

    return out

# ---------------------------------------------------------------------
# 실행
# ---------------------------------------------------------------------
parsed = df["description"].apply(parse_description)
parsed_df = pd.DataFrame(parsed.tolist())

# 원본과 합치고 저장 (Excel 호환을 위해 utf-8-sig 권장)
final_df = pd.concat([df, parsed_df], axis=1)
final_df.to_csv("./dataset/separated_perfume.csv", index=False, encoding="utf-8-sig")

print(final_df[["부향률", "메인 어코드", "탑 노트", "미들 노트", "베이스 노트", "향 설명"]].head(3))

      부향률           메인 어코드                         탑 노트               미들 노트  \
0  오 드 퍼퓸     프루티, 스위트, 레더  베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼  파인애플, 패출리, 모로칸 자스민   
1  오 드 퍼퓸     프루티, 스위트, 레더  베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼  파인애플, 패출리, 모로칸 자스민   
2  오 드 퍼퓸  우디, 오우드, 웜 스파이시                                                    

                         베이스 노트  \
0  자작나무, 머스크, 오크 모스, 암브록산, 시더우드   
1  자작나무, 머스크, 오크 모스, 암브록산, 시더우드   
2                                 

                                                향 설명  
0  용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...  
1  용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...  
2             청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향  


In [13]:
# 공지 문구 제거
import re

# NOTICE_TEXT = "일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다. 이로 인한 교환/반품은 어려운 점 참고 부탁드립니다."
NOTICE_TEXT = "본 제품은 아시아에서 유통되는 상품이 아닌, 유럽에서 유통되는 상품입니다."

def clean_notice(desc: str) -> str:
    """향 설명에서 공지 문구 제거"""
    if not isinstance(desc, str):
        return desc
    # 지정된 공지 문구 제거
    cleaned = desc.replace(NOTICE_TEXT, "")
    # 불필요한 공백 정리
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

# 적용
final_df["향 설명"] = final_df["향 설명"].apply(clean_notice)

# 저장
final_df.to_csv("./dataset/separated_perfume_cleaned.csv", index=False, encoding="utf-8-sig")

print(final_df["향 설명"].head(3))

0             용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향
1             용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향
2    청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
Name: 향 설명, dtype: object


In [14]:
import pandas as pd

df = pd.read_csv("./dataset/separated_perfume_cleaned.csv", encoding="utf-8-sig")
df.head()

,brand,name,size_ml,price_krw,detail_url,description,부향률,메인 어코드,탑 노트,미들 노트,베이스 노트,향 설명
0,크리드,어벤투스 오 드 퍼퓸,50ml,255000.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향"
1,크리드,어벤투스 오 드 퍼퓸,100ml,399220.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향"
2,톰 포드,오드 우드 오 드 퍼퓸,30ml,179000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
3,톰 포드,오드 우드 오 드 퍼퓸,50ml,249000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
4,이솝,테싯 오 드 퍼퓸,50ml,135000.0,https://www.bysuco.com/product/show/9970,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 아로마틱 / ...,오 드 퍼퓸,"시트러스, 아로마틱, 프레쉬 스파이시","유자, 시트러스",바질,"베티버, 클로브",이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향
